# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [7]:
import warnings

In [8]:
warnings.filterwarnings("ignore")

In [6]:
import gensim
import pandas as pd
import numpy as np
import itertools
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [12]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [5]:
data = pd.read_csv('avito_category_classification.csv')

In [6]:
# # test

data = data[:5000]

In [7]:
data['description_norm'] = data['description'].apply(normalize)

In [8]:
len(data)

5000

### Сначала рассмотрим SVD

In [9]:
pipeline_svd_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier())
])

pipeline_svd_kneir = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier())
])

pipeline_svd_rf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])

pipeline_svd_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=10))
])


In [10]:
def eval_table(X, y, pipeline, N=6):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов будет хранить в табличке
    fold_metrics = pd.DataFrame(index=labels)
    # дополнительно также соберем таблицу ошибок
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    # shuffle=True (перемешивание) - часто критично важно указать
    # т.к. данные могут быть упорядочены и модель на этом обучится
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        # fit-predict как и раньше, но сразу пайплайном
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    # тут мы берем колонки со значениями и усредняем их
    # часто также все метрики сразу суммируют и в конце просто делят на количество фолдов
    # но мы тут помимо среднего также хотим посмотреть на стандартное отклонение
    # чтобы понять как сильно варьируются оценки моделей
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

In [11]:
metrics_svd_sgd, errors_svd_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_svd_sgd)
metrics_svd_kneir, errors_svd_kneir = eval_table(data['description_norm'], data['category_name'], pipeline_svd_kneir)
metrics_svd_rf, errors_svd_rf = eval_table(data['description_norm'], data['category_name'], pipeline_svd_rf)
metrics_svd_etc, errors_svd_etc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_etc)

In [12]:
metrics_svd_sgd

,precision,precision_std,recall,recall_std,f1,f1_std
Телефоны,0.77,0.10,0.73,0.05,0.75,0.03
"Одежда, обувь, аксессуары",0.69,0.01,0.78,0.04,0.73,0.02
Детская одежда и обувь,0.75,0.05,0.73,0.02,0.74,0.03
Ремонт и строительство,0.51,0.13,0.41,0.07,0.45,0.09
Мебель и интерьер,0.65,0.06,0.59,0.08,0.62,0.06
Товары для детей и игрушки,0.69,0.08,0.60,0.09,0.64,0.08
Бытовая техника,0.52,0.08,0.43,0.14,0.46,0.11
Квартиры,0.96,0.02,0.95,0.02,0.95,0.02
Предложение услуг,0.73,0.06,0.76,0.07,0.74,0.04
Автомобили,0.80,0.11,0.88,0.02,0.84,0.06


In [13]:
res = pd.DataFrame(columns=list(itertools.chain(["model"], metrics_svd_rf.columns)) )

In [14]:
res.loc[0] = (list(['SGDClassifier svd'] + list(metrics_svd_sgd.loc['mean'].values)))
res.loc[1] = (list(['KNeighborsClassifier svd'] + list(metrics_svd_kneir.loc['mean'].values)))
res.loc[2] = (list(['RandomForest svd'] + list(metrics_svd_rf.loc['mean'].values)))
res.loc[3] = (list(['ExtraTreesClassifier svd'] + list(metrics_svd_etc.loc['mean'].values)))
res

,model,precision,precision_std,recall,recall_std,f1,f1_std
0,SGDClassifier svd,0.71,0.07,0.69,0.06,0.69,0.05
1,KNeighborsClassifier svd,0.48,0.07,0.39,0.07,0.41,0.06
2,RandomForest svd,0.75,0.14,0.36,0.05,0.39,0.06
3,ExtraTreesClassifier svd,0.67,0.22,0.21,0.03,0.20,0.04


### Теперь рассмотрим NMF

In [15]:
pipeline_nmf_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('decomposition', NMF(100)),
    ('clf', SGDClassifier())
])

pipeline_nmf_kneir = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('decomposition', NMF(100)),
    ('clf', KNeighborsClassifier())
])

pipeline_nmf_rf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('decomposition', NMF(100)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])

pipeline_nmf_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.6)),
    ('decomposition', NMF(100)),
    ('clf', ExtraTreesClassifier(n_estimators=100, max_depth=10))
])


In [16]:
metrics_nmf_sgd, errors_nmf_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_sgd)
metrics_nmf_kneir, errors_nmf_kneir = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_kneir)
metrics_nmf_rf, errors_nmf_rf = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_rf)
metrics_nmf_etc, errors_nmf_etc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_etc)

In [17]:
res.loc[4] = (list(['SGDClassifier nmf'] + list(metrics_nmf_sgd.loc['mean'].values)))
res.loc[5] = (list(['KNeighborsClassifier nmf'] + list(metrics_nmf_kneir.loc['mean'].values)))
res.loc[6] = (list(['RandomForest nmf'] + list(metrics_nmf_rf.loc['mean'].values)))
res.loc[7] = (list(['ExtraTreesClassifier nmf'] + list(metrics_nmf_etc.loc['mean'].values)))

In [18]:
res

,model,precision,precision_std,recall,recall_std,f1,f1_std
0,SGDClassifier svd,0.71,0.07,0.69,0.06,0.69,0.05
1,KNeighborsClassifier svd,0.48,0.07,0.39,0.07,0.41,0.06
2,RandomForest svd,0.75,0.14,0.36,0.05,0.39,0.06
3,ExtraTreesClassifier svd,0.67,0.22,0.21,0.03,0.20,0.04
4,SGDClassifier nmf,0.60,0.20,0.48,0.11,0.48,0.08
5,KNeighborsClassifier nmf,0.42,0.08,0.37,0.06,0.38,0.06
6,RandomForest nmf,0.69,0.12,0.51,0.05,0.54,0.05
7,ExtraTreesClassifier nmf,0.71,0.16,0.35,0.05,0.38,0.06


### Результат:

Лучшее сочетание по критерию средней F1 меры - это SVD разложение и SGD классификатор.

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

Нграммы добавляются вот так (перед созданиеv словаря)

In [19]:
# texts = [text.split() for text in texts]
# ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
# p = gensim.models.phrases.Phraser(ph)
# ngrammed_texts = p[texts]  

# ! не забудьте, что далее вам нужно будет использовать ngrammed_texts

!! В модели с нграммами вначале посмотрите, что получается после преобразования
Если вы выведите несколько первых текстов в ngrammed_texts, то там должно быть что-то такое:

In [20]:
# [text for text in ngrammed_texts[:3]]
# >> [['новостройка',
#   'нижегородский_область', # нграм
#   'новостро́йка',
#   '—',
#   'сельский',
#   'посёлок',
#   'в',
#   'дивеевский_район', # нграм
#   'нижегородский_область', #нграмм
#   'входить',
#   'в',
#   'состав_сатисский', #нграмм
#   'сельсовет',
#   'посёлок',
#   'расположить',
#   'в',
#   '12,5',
#   'километр',
# ....

Если вы не видите нграммов, то попробуйте изменить параметр threshold

Tfidf добавляется вот так (после векторизации и перед обучением lda)

In [21]:
# tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, )
# corpus = tfidf[corpus]

### Загрузим и предобработаем тексты

In [9]:
num_topics = 30

In [13]:
texts = open('wiki_data.txt', encoding="utf8").read().splitlines()[:1000]

In [14]:
texts = ([normalize(text) for text in texts])

In [15]:
len(texts)

1000

In [54]:
texts = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

### Функция для получения метрик

In [55]:

def get_metrics(model, corpus, texts, dictionary):
    topics = []
    for topic_id, topic in model.show_topics(num_topics=num_topics, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics,
                                                       texts=texts,
                                                       dictionary=dictionary, coherence='c_v')
    
    return np.exp2(-model.log_perplexity(corpus)), coherence_model_lda.get_coherence()

### Стандартная модель

In [56]:
dictionary = gensim.corpora.Dictionary((text for text in texts))
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in texts]

lda_1 = gensim.models.LdaMulticore(corpus, 
                                 num_topics, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)

In [58]:
res = get_metrics(lda_1, corpus, texts, dictionary)
print(f"Перплексия: {res[0]}\nКогерентность: {res[1]}")

Перплексия: 188.5555754110799
Когерентность: 0.4314441933977406


### TfIdf

In [59]:
dictionary = gensim.corpora.Dictionary((text for text in texts))
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
tf_idf_corpus = tfidf[corpus]

lda_tfidf = gensim.models.LdaMulticore(tf_idf_corpus,
                                        num_topics,
                                        alpha='asymmetric',
                                        id2word=dictionary,
                                        passes=10)

In [60]:
res = get_metrics(lda_tfidf, tf_idf_corpus, texts, dictionary)
print(f"Перплексия: {res[0]}\nКогерентность: {res[1]}")

Перплексия: 1142.130824311733
Когерентность: 0.4206214052183683


### Ngrams

In [61]:
dictionary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictionary.filter_extremes(no_above=0.05, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

lda_ngrams = gensim.models.LdaMulticore(corpus, 
                                     num_topics, # колиество тем
                                     alpha='asymmetric',
                                     id2word=dictionary, 
                                     passes=10) 

In [62]:
res = get_metrics(lda_ngrams, corpus, ngrammed_texts, dictionary)
print(f"Перплексия: {res[0]}\nКогерентность: {res[1]}")

Перплексия: 231.5170435138144
Когерентность: 0.45741584129128954


### TfIdf + Ngrams

In [63]:
dictionary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictionary.filter_extremes(no_above=0.05, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, smartirs='lfc')
corpus = tfidf[corpus]

lda_tfidf_ngrams = gensim.models.LdaMulticore(corpus, 
                                 num_topics, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10) 

In [64]:
res = get_metrics(lda_tfidf_ngrams, corpus, ngrammed_texts, dictionary)
print(f"Перплексия: {res[0]}\nКогерентность: {res[1]}")

Перплексия: 1163.4630347492553
Когерентность: 0.3959155196392948


### Анализ выделенных тем

In [80]:
alph = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя "

def get_words(arr: list):
    return "\n\n".join(["".join([c for c in x[1] if c in alph]) for x in arr])

Стандартная модель:

In [81]:
print(get_words(lda_1.print_topics()[:]))

премия  китайский  писатель  произведение  роман  н  язык  литература  э  литературный

залив  река    связь  структура  побережье  впадать  значительный  берег  долина

система  устройство  случай  плод  данные  например  ребёнок  суд    объект

орудие  фраза  музыкальный  мотив  предложение  установка    вариант  основный  период

й  аэропорт  значение    пространство    й  международный    

германия  польский  памятник  проект  песня  берлин  мария  искусство  норвежский  фестиваль

лагерь  газета  заключить    тысяча    политический  здание  среди  убить

альбом  клуб  лига  команда  выпустить  песня  играть  чемпионат    матч

клуб  команда  футбольный    чемпионат  сезон  участник  среди  соревнование  фамилия

университет  колледж  соревнование  список  факультет  объект  образование  студент  канада  отделение

штат  сезон  собрание  клуб  национальный  команда  финал  компания  конституция  серия

фильм  роль  брат  жизнь  ребёнок  вместе  главный  театр  отец  чтобы

  самол

Осмысленная тема - альбом  клуб  лига  команда  выпустить  песня  играть  чемпионат. Про многие топики можно сказать, что сложно сходу определить их тему.


TfIdf

In [82]:
print(get_words(lda_tfidf.print_topics()[:]))

матч  джордж  зелёный  ирландия  московский  университет  й  турнир  голос  заслуга

конечный  круглый  изобразить  историк  каменный  комиссар  конгресс  здание  красивый  лишить

индия  выставка  федеральный  швейцария  всемирный  суд  заметный  похожий  художник  статистика

королевский  лондонский  церемония  должный  офицер  морской  есть  вооружённый  король  век

конечный  круглый  изобразить  историк  каменный  комиссар  конгресс  здание  красивый  лишить

бассейн  дворец  правитель  невозможно  упасть  крест  избежать  соорудить  главный  фигура

вмс  военноморской  эсминец  тип  китайский  база  эксплуатация  честь    американский

театр  роль  сергей  музыкальный  компания  шоу  театральный  санктпетербург  играть  руководство

роль  сыграть  телесериал  сняться  фильм  сниматься  пилот    эпизод  постановка

цикл  код  параметр  факт  стоять    случай  команда  производить  вызывать

существо  живой  концепция  конституция  посёлок  республика  остров  главное  азия  полном

Красивая тема - королевский  лондонский  церемония  должный  офицер  морской  есть  вооружённый  король  век. Кажется, что эта модель получше, чем предыдущая.

Ngrams

In [83]:
print(get_words(lda_ngrams.print_topics()[:]))

корпус  дивизия  командир  полковник  командующий  операция  соединение  фронт  генералмайор  полка

завод  посёлок  предприятие  производить    финансовый  совет  жена  париж  цикл

клуб  матч  сборная  сезон  выиграть  лига  кубок  чемпионат  сыграть  игрок

харьковскийобласть  нарасстояние  поперепись  мж  сельскийсовет  сезон  ниже  кодкоатуа  расположитьсело  шевченковскийрайон

здание  подразделение  участок  иван  н  позиция  бой  немецкий  командование  отряд

объект  передача  здание  культура  церковь  передать  собственность  российскийфедерация  закон  председатель

залив  вода  применение  например  обычно  применяться  произойти  найти  поэтому  термин

список  дворец  архитектор  здание  деревянный  ворота  строительство  архитектура  пётр  музей

игрок  выпустить    разработать  доступный  австралия  режим  персонаж  образ  версия

пара  джордж  ирландия  король  иногда    говорить  пол  правило  начаться

эсминец  произвести  сражение  флот  корабль  камень  полный  до

Мне понравилась тема - корпус  дивизия  командир  полковник  командующий  операция  соединение  фронт  генералмайор  полка. По когерентности лучше всех эта модель. Качество тем довольно таки хорошее.

TfIdf + Ngrams

In [84]:
print(get_words(lda_tfidf_ngrams.print_topics()[:]))

памятник  германия  городской  линия  собор  польский  берлин  маршрут  поместить  сеть

простой  дон  дерево  э    конструкция  вооружение  оружие  царь  эпоха

курс  первыйсекретарь    организовать  сотрудничатьс  депутатверховный  кпсс  считатьчто  николаевич  созыв

соединение  анализ  определение  метод  изучение  академиянаука  учёный  завод  л  исследование

анатолий  тренер  первенство  бронзовыйпризёр  фестиваль  сборная  женский    чемпионмир  

владимир  майор  я  париж  вы  смочь  состояние  найти  расследование  самоубийство

курс  первыйсекретарь    организовать  сотрудничатьс  депутатверховный  кпсс  считатьчто  николаевич  созыв

курс  первыйсекретарь    организовать  сотрудничатьс  депутатверховный  кпсс  считатьчто  николаевич  созыв

курс  первыйсекретарь    организовать  сотрудничатьс  депутатверховный  кпсс  считатьчто  николаевич  созыв

кнр  свидетель  китай  китайский  религиозный  политика  практика  религия  приказ  свобода

успеть  сценарий  великийотечествен

Понравилась тема - анатолий  тренер  первенство  бронзовыйпризёр  фестиваль  сборная  женский    чемпионмир  . Странно, что 3 раза повторяется один и тот же набор слов (курс  первыйсекретарь    организовать  сотрудничатьс  депутатверховный  кпсс  считатьчто  николаевич  созыв) - так сработал алгоритм. Кажется, эта модель на глаз точно не лучше 2 предыдущих.

Думаю, что модель ngrams лучше всех - мне так показалось на глаз. По метрикам выигрывает стандартный алгоритм и n-граммы. 